# Design Experiment in AML
The following illustrates examples of how to use AML to perform day to day machine learning task.

In [ ]:
subscription_id = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
resource_group = "RESOURCE_GROUP_NAME"
workspace = "WORKSPACE_NAME"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION=subscription_id
RESOURCE_GROUP=resource_group
WS_NAME=workspace

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# Verify that the handle works correctly.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

## Execute training locally
The following example shows how to leverage AML as cloud development machine. Training is done locally in Juptyer notebook or via python script.

The job is executed in compute instance.

In [ ]:
!pip install pandas numpy scikit-learn joblib

In [ ]:
# Read titanic data

import mltable

titanic_version = 'xxxx'

data_asset = ml_client.data.get(name="titanic-cloud-example", version=titanic_version)

# create a table
tbl = mltable.load(f"azureml:/{data_asset.id}")

# load into pandas
df = tbl.to_pandas_dataframe()
df.head(5)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Convert categorical columns to numerical
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

# Define features and target
X = df.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin'])
y = df['Survived']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a logistic regression classifier
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# Save the model and scaler
joblib.dump(clf, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Evaluate the classifier
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

In [ ]:
import joblib
import numpy as np

# Load the model and scaler
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')

data = {"Pclass": 3, "Age": 22, "SibSp": 1, "Parch": 0, "Fare": 7.25, "Sex_male": 1, "Embarked_Q": 0, "Embarked_S": 1}

#features = np.array([data['Pclass'], data['Age'], data['SibSp'], data['Parch'], data['Fare'], data['Sex_male'], data['Embarked_Q'], data['Embarked_S']])
feature_names = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']
input_data = pd.DataFrame([data], columns=feature_names)
# Scale the features
scaled_features = scaler.transform(input_data)
# Make a prediction
prediction = model.predict(scaled_features)

print(f'Predicted result: {prediction[0]}')